# Fasttext

Fasttext er et en teknikk som representerer ord som vektorer, og bruker shallow neural networks til smart tekstanalyse. Det er utviklet av Facebook som et lett alternativ til deep neural networks som trenger kraftig maskinvare. 
Fasttext gir omtrent like god ytelse som tf-idf og support vector machines, gensim/word2vec, eller deep neural networks. 

Følg [instruksjoner](https://fasttext.cc/docs/en/support.html) for å komme igang. 

In [ ]:
!mkdir fasttext
!mkdir data
!mkdir fasttext/data

In [ ]:
import os
import sys


data_dir = os.path.join('.', 'data')

print(sys.version)

In [ ]:
from lib.data import download_data

download_data(data_dir)

In [ ]:
from lib.load_dataset import load_datasets

df = load_datasets(data_dir, 2015, 2015)

print(df.shape)
df.head()

In [ ]:
import re
from lib.preprocessing import remove_tags, unescape_html, remove_whitespace

non_alphanumeric_pattern = re.compile(r'\W')

# Select subset of columns
col = [
    'stillingsbeskrivelse', 
    'yrke_grovgruppe',
    'stilling_kilde',
]
df = df[col]


# Preprocess text and add some columns
df['stillingsbeskrivelse'] = df['stillingsbeskrivelse']\
    .map(remove_tags)\
    .map(unescape_html)\
    .map(remove_whitespace)\
    .map(lambda x: non_alphanumeric_pattern.sub(' ', x))\
    .map(lambda x: x.lower())
df['is_from_nav'] = df['stilling_kilde'].map(lambda x: x=='Reg av arb.giver på nav.no')
df['from_media'] = df['stilling_kilde'].map(lambda x: x=='Annonsert i media')
df['is_healthcare'] = df['yrke_grovgruppe'].map(lambda x: x=='Helse, pleie og omsorg')
df['is_industrial'] = df['yrke_grovgruppe'].map(lambda x: x=='Industriarbeid')

print(df.shape)
df.head()




In [ ]:
import csv
from sklearn.model_selection import train_test_split

label_column = 'yrke_grovgruppe'

def make_dataframe_fasttext_format(df, label_col, size=None, random_seed=305):
    if not size:
        size = len(df)
    result = df.sample(n=size, random_state=random_seed)[[label_col, 'stillingsbeskrivelse']]
    result[label_col] = result[label_col].map(lambda x: unicode(x).replace(' ', '_'))
    result[label_col] = result[label_col].map(lambda x: '__label__' + unicode(x))
    return result


out = make_dataframe_fasttext_format(df, label_column)

def write_train_test_files_for_fasttext(df, name, test_frac=0.2):
    train_df, test_df = train_test_split(df, test_size=test_frac)
    test_df.to_csv(os.path.join('.', 'fasttext', 'data', ''.join([name, '.test'])), escapechar=' ', sep=' ', header=None, mode='w', index=False, quoting=csv.QUOTE_NONE, encoding='utf-8')
    train_df.to_csv(os.path.join('.', 'fasttext', 'data', ''.join([name, '.train'])), escapechar=' ', sep=' ', header=None, mode='w', index=False, quoting=csv.QUOTE_NONE, encoding='utf-8')
    

write_train_test_files_for_fasttext(out, label_column)